# Problem overview

Given two frames $\{M\}$ and $\{F\}$, suppose a wrench $(f, m)$ acts on the origin
of frame $\{F\}$, what is the equivalent wrench at frame $\{M\}$?

We first look at the force and moment as pure vectors without coordinate. Let $f', m'$
be the equivalent force and moment at frame $\{M\}$, one has the following equation

$$
f' = f \\
m' = m + p \times f
$$

where $p$ is the vector originating from the origin of frame $\{F\}$ and ending at the origin
of frame $\{M\}$.

These equations follow directly from force and moment-equilibrium equations.

In order to carry out numerical computations, it is necessary to re-write these
equations as coordinates in some coordinates system. Notice that the coordinates of
$(f, m)$ in frame $\{F\}$, we can start by re-writing the right hand-side in this
coordinate. Indeed, let $f_F, p_F, m_F$ denote respectively column vectors containing
the coordinates, one has that:

$$
f_F \\
m_F + p_F \times f_F
$$

We have obtain the coordiantes of $(f', m')$ in frame $\{F\}$. What's left to be done
is to convert these to coordinates in frame $\{M\}$. This is done easily using the rotation
matrix $R_{MF}$: The axes of frames $F$ in coordinate system of frame $M$. Hence one has:

$$
R_{MF} f_F \\
R_{MF} (m_F + p_F \times f_F)
$$

Rearranging the term, we can derive a linear mapping from wrench coordinates directly:

$$
\begin{bmatrix} R_{MF} & 0 \\ R_{MF} [p_F] & R_{MF}
\end{bmatrix}
$$

# Code

Nothing beats some code. We now look at some code example.

In [2]:
import numpy as np

def wrench_transform(transform_FM):
    """Return the 6x6 wrench transform.
    """
    
    R_FM = transform_FM[:3, :3]
    p_FM = transform_FM[:3, 3]
    R_MF = R_FM.T
    
    # https://en.wikipedia.org/wiki/Cross_product
    a1, a2, a3 = p_FM
    p_FM_cross = np.array([[0, -a3, a2],
                           [a3, 0, -a1],
                           [-a2, a1, 0]])
    
    wrench_mapping = np.zeros((6, 6))
    wrench_mapping[:3, :3] = R_MF
    wrench_mapping[3:, :3] = np.dot(R_MF, p_FM_cross)
    wrench_mapping[3:, 3:] = R_MF
    return wrench_mapping

## Example 1: M and F coincide

the mapping should be the identity matrix.

In [4]:
transform_FM = np.eye(4)
mapping = wrench_transform(transform_FM)
print(mapping)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]


## Example 2: F is 2 along the y-axis of M


In [8]:
transform_FM = np.eye(4)
transform_FM[:3, 3] = [0, 2, 0]

mapping = wrench_transform(transform_FM)
print(mapping)

wrench = [-1, 0, 0, 0, 0, 0]
print(np.dot(mapping, wrench))

[[ 1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  2.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [-2.  0.  0.  0.  0.  1.]]
[-1.  0.  0.  0.  0.  2.]
